In [1]:
def acked(err, msg):
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg.value()), str(err)))
    else:
        print("Message produced: %s" % (str(msg.value())))

In [2]:
from socket import gethostname
from pandas import read_csv
from sys import exit
from confluent_kafka import Producer
TOPIC_NAME = "review_stream"
KAFKA_SERVER = "kafka:9092" 
CSV_FILE_PATH = "test_data.csv" 
DELAY_SECONDS = 1

# Các cột nhãn gốc từ file CSV
ASPECT_COLUMNS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']

conf = {'bootstrap.servers': KAFKA_SERVER,
        'client.id': gethostname()}
producer = Producer(conf)

try:
    df = read_csv(CSV_FILE_PATH)
    required_columns = ['Review'] + ASPECT_COLUMNS
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"LỖI: Thiếu các cột sau trong {CSV_FILE_PATH}: {', '.join(missing_cols)}")
        exit(1)       
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file {CSV_FILE_PATH}")
    exit(1)
except Exception as e:
    print(f"Lỗi khi đọc CSV: {e}")
    exit(1)

%3|1761537382.727|FAIL|278def14d2e3#producer-1| [thrd:kafka:9092/bootstrap]: kafka:9092/bootstrap: Connect to ipv4#172.19.0.3:9092 failed: Connection refused (after 1ms in state CONNECT)


In [3]:
from pandas import isna
from json import dumps
from time import sleep
try:
    for index, row in df.iterrows():
        review_text = row['Review']
        if isna(review_text):
            print("Bỏ qua bình luận rỗng (NaN)")
            continue
            
        payload = {
            "review_text": str(review_text)
        }
        for aspect in ASPECT_COLUMNS:
             label_value = row[aspect]
             if isna(label_value):
                 payload[aspect] = None
             else:
                 payload[aspect] = int(label_value)


        message_value = dumps(payload).encode('utf-8')
        
        producer.produce(TOPIC_NAME, value=message_value, callback=acked)
        
        producer.poll(0)
        
        # Đợi 1 giây
        print("By 23521198 - Lê Nguyễn Hoàng Phúc")
        sleep(DELAY_SECONDS)

except KeyboardInterrupt:
    print("Đã dừng bởi người dùng.")
finally:
    # Đảm bảo tất cả tin nhắn trong hàng đợi đều được gửi
    print("Đang xả (flushing) hàng đợi producer...")
    producer.flush()
    print("Hoàn tất.")

%3|1761537383.841|FAIL|278def14d2e3#producer-1| [thrd:kafka:9092/bootstrap]: kafka:9092/bootstrap: Connect to ipv4#172.19.0.3:9092 failed: Connection refused (after 34ms in state CONNECT, 1 identical error(s) suppressed)


By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
By 23521198 - Lê Nguyễn Hoàng Phúc
Message produced: b'{"review_text": "Gi\\u00e0y h\\u01a1i c\\u00f3 m\\u00f9i n\\u1ed3ng, l\\u01b0u \\u00fd \\u0111\\u00f4i LA kh\\u00f4ng ph\\u1ea3i \\u0111\\u1ebf x\\u00e1m n\\u00ean mng c\\u00e2n nh\\u1eafc k\\u0129 nh\\u00e9 \\u1ea1 nhma v\\u1edbi gi\\u00e1 ti\\u1ec1n n\\u00e0y th\\u00ec l\\u00e0 oke", "Price": -1, "Shipping": -1, "Outlook": -1, "Quality": 2, "Size": -1, "Shop_Service": -1, "General": 2, "Others": -1}'
Message produced: b'{"review_text": "H\\u00e0ng v\\u1ec1 \\u0111\\u1eb9p l\\u1eafm nha ship th\\u00e2n thi\\u1ec7n \\u0111i gi\\u00e0y v\\u1eeba in\\u2764\\ufe0f", "Price": -1, "Shipping": 